In [1]:
import sys
import os

sys.path.append("..")

DATABASE_DIR_MANIFESTOS = "../../data/manifestos/chroma/openai"
DATABASE_DIR_DEBATES = "../../data/manifestos/chroma/openai"
TEST_DATA_DIR = "../../data/questions"

# Here we define the experimental run! This is important for the filenames, etc!

In [2]:
experiment_name = "1_anonymize_parties"
experiment_run_name = "1a_parties_not_anonymous"

In [3]:
# Function to deanonymize parties
def rename_party(party: str, mode: str = "anonymize"):
    """
    Anonymizes or de-anonymizes party names.

    arguments:
    - party (str): party name
    - mode (str): "anonymize" or "deanonymize"

    returns:
    - party (str): anonymized party name
    """

    party_dict = {
        "cdu": "partei_a",
        "spd": "partei_b",
        "gruene": "partei_c",
        "linke": "partei_d",
        "fdp": "partei_e",
        "afd": "partei_f",
    }

    if mode == "anonymize":
        return party_dict[party.lower()]
    elif mode == "deanonymize":
        for key, value in party_dict.items():
            if value == party.lower():
                return key
    else:
        raise ValueError("mode must be 'anonymize' or 'deanonymize'")

In [4]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm


from database.vector_database import VectorDatabase
from models.embedding import ManifestoBertaEmbeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_mistralai.chat_models import ChatMistralAI


from datasets import Dataset, DatasetDict, load_from_disk

# Create chain

In [5]:
from models.generation import generate_chain_with_balanced_retrieval

# embedding_model = ManifestoBertaEmbeddings()
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

LARGE_LANGUAGE_MODEL = ChatOpenAI(
    model_name="gpt-3.5-turbo", max_tokens=2000, temperature=0
)

# LARGE_LANGUAGE_MODEL = ChatMistralAI(
#     name="open-mixtral-8x7b", max_tokens=2000, temperature=0
# )

db_manifestos = VectorDatabase(
    data_path="../data/manifestos",
    embedding_model=embedding_model,
    database_directory=DATABASE_DIR_MANIFESTOS,
    source_type="manifestos",
)

chain = generate_chain_with_balanced_retrieval(
    [db_manifestos],
    llm=LARGE_LANGUAGE_MODEL,
    return_context=True,
    language="Deutsch",
    k=5,
)

reloaded database


# Create dataset with question, context, and answer for chain

In [6]:
recreate_dataset = False
if recreate_dataset:
    # Create dataset with question, context and answer

    # Load all test questions
    df_test_simple = pd.read_csv(os.path.join(TEST_DATA_DIR, "simple_questions.csv"))
    df_test_complex = pd.read_csv(os.path.join(TEST_DATA_DIR, "complex_questions.csv"))

    # Here we select, which parties we want to evaluate
    parties = ["afd", "spd", "linke", "gruene", "fdp", "cdu"]

    # Here we select the questions that should be evaluated
    questions = df_test_simple["EINFACHE FRAGEN"]

    # Dictionary to save results
    dataset_dict = {
        party: {"question": [], "contexts": [], "answer": []} for party in parties
    }

    # Loop through all questions and get chain answer
    for question in tqdm(questions):

        max_retries = 4
        retry_count = 0
        while retry_count <= max_retries:
            try:
                response = chain.invoke(question)
                assert set(response["answer"].keys()) == set(parties)
                break
            except:
                retry_count += 1
                print(f"Exception occurred, retry_count = {retry_count}")

            # response["answer"] = {rename_party(party, mode = "deanonymize"): response["answer"][party] for party in response["answer"].keys()}
            # response["answer"] = {rename_party(party, mode = "deanonymize"): response["answer"][party] for party in response["answer"].keys()}

        # for pary in parties:
        for party in parties:
            contexts = [
                doc.page_content for doc in response["docs"]["manifestos"][party]
            ]
            answer = response["answer"][party]

            dataset_dict[party]["question"].append(question)
            dataset_dict[party]["contexts"].append(contexts)
            dataset_dict[party]["answer"].append(answer)

    dataset = DatasetDict(
        {party: Dataset.from_dict(dataset_dict[party]) for party in parties}
    )

    # Save
    dataset.save_to_disk(f"Experiments/{experiment_name}/dataset_{experiment_run_name}")

In [7]:
dataset = load_from_disk(
    f"Experiments/{experiment_name}/dataset_{experiment_run_name}")

# Import Ragas and translate prompts to German

In [8]:
import ragas
from ragas.metrics import faithfulness, answer_relevancy, context_relevancy


# llm used for adaptation
openai_model_adaption = ChatOpenAI(model_name="gpt-4")
ragas.adapt(
    metrics=[faithfulness, answer_relevancy, context_relevancy],
    language="german",
    llm=openai_model_adaption,
)

# Compute metrics with ragas

In [10]:
from ragas.run_config import RunConfig

# Configuration for ragas.evaluate
run_config = RunConfig()
run_config.max_retries = 3
run_config.max_timeout = 60  # in s
run_config.max_wait = 20  # in s

num_questions = 28
questions = [dataset["fdp"][i]["question"] for i in range(num_questions)]

results_dict = {}
parties = ["afd", "spd", "linke", "gruene", "fdp", "cdu"]
list_of_metrics = [faithfulness, answer_relevancy]

for i in tqdm(range(num_questions)):
    results_dict.update({questions[i]: {}})
    print(f"Question: {questions[i]}")
    for party in parties:
        print(f"Evaluating party: {party}")
        ragas_score = ragas.evaluate(
            dataset[party].select([i]), metrics=list_of_metrics, run_config=run_config
        )
        results_dict[questions[i]].update({party: ragas_score})
    print(results_dict[questions[i]])

  0%|          | 0/28 [00:00<?, ?it/s]

Question: Was denken die Parteien darüber, mehr Geld für die Armee auszugeben, um Europa sicher zu halten?
Evaluating party: afd


Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluating party: spd


Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluating party: linke


Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluating party: gruene


Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Flatten the dictionary
flattened_data = {}
for question, parties in results_dict.items():
    for party, metrics in parties.items():
        for metric, value in metrics.items():
            flattened_data[(question, party, metric)] = value

# Create a multi-index DataFrame
index = pd.MultiIndex.from_tuples(flattened_data.keys())
df = pd.Series(flattened_data, index=index).unstack().unstack()

file_name = f"Experiments/{experiment_name}/metrics_{experiment_run_name}.csv"

df.to_csv(file_name)

In [ ]:
# Check if file saved correctly
df = pd.read_csv(file_name, header=[0, 1], index_col=[0, 1])
df.head()